# **IMPORTING LIBRARIES**

In [1]:
import os
import yaml
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import cv2
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, jaccard_score
from functools import partial
import torch.utils.checkpoint as checkpoint
import warnings
import random
warnings.filterwarnings("ignore")

In [2]:
# Add at the top of the file after imports
torch.backends.cuda.max_memory_reserved = True
torch.backends.cudnn.benchmark = True

# Add these environment variables
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:32'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['TORCH_USE_CUDA_DSA'] = '1'

In [3]:
# Check CUDA setup
def check_cuda():
    """Check and print CUDA information once"""
    print("\n=== CUDA Setup ===")
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    print(f"CUDA device count: {torch.cuda.device_count()}")
    if torch.cuda.is_available():
        print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
    print("=================\n")

# Call check_cuda once
check_cuda()

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


=== CUDA Setup ===
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA device count: 1
CUDA device name: Tesla P100-PCIE-16GB



# **BEiT Backbone Implementation**

In [4]:
# Add this before creating the datasets
import glob

# Check the actual directory structure
def check_directory_structure(base_dir):
    print(f"\n--- Directory Structure Check for {base_dir} ---")
    
    # Check main directories
    train_img_dir = os.path.join(base_dir, "train", "images")
    train_mask_dir = os.path.join(base_dir, "train", "labels")
    val_img_dir = os.path.join(base_dir, "valid", "images")
    val_mask_dir = os.path.join(base_dir, "valid", "labels")
    
    for dir_path in [train_img_dir, train_mask_dir, val_img_dir, val_mask_dir]:
        exists = os.path.exists(dir_path)
        if exists:
            files = glob.glob(os.path.join(dir_path, "*"))
            file_count = len(files)
            print(f"Directory {dir_path}: {'Exists' if exists else 'MISSING'}, Contains {file_count} files")
            if file_count > 0:
                # Show some example files
                print(f"  Example files: {[os.path.basename(f) for f in files[:3]]}")
        else:
            print(f"Directory {dir_path}: MISSING")
    
    # Try to find the first image and its corresponding mask
    if os.path.exists(train_img_dir) and os.path.exists(train_mask_dir):
        img_files = glob.glob(os.path.join(train_img_dir, "*.jpg")) + \
                    glob.glob(os.path.join(train_img_dir, "*.png"))
        
        if img_files:
            # Get first image
            first_img = img_files[0]
            img_basename = os.path.splitext(os.path.basename(first_img))[0]
            
            # Look for matching mask
            potential_masks = [
                os.path.join(train_mask_dir, f"{img_basename}.txt"),
                os.path.join(train_mask_dir, f"{img_basename}.png"),
                os.path.join(train_mask_dir, f"{img_basename}.jpg")
            ]
            
            found_mask = None
            for mask_path in potential_masks:
                if os.path.exists(mask_path):
                    found_mask = mask_path
                    break
            
            print(f"\nMatching check:")
            print(f"  Image: {first_img}")
            print(f"  Mask: {found_mask if found_mask else 'NO MATCHING MASK'}")
            
            if found_mask and found_mask.endswith('.txt'):
                # Show the content of the txt mask
                with open(found_mask, 'r') as f:
                    mask_content = f.read()
                print(f"\nMask content (first 200 chars):\n{mask_content[:200]}")

In [5]:
def drop_path(x, drop_prob: float = 0., training: bool = False):
    """Drop paths (Stochastic Depth) per sample (when applied in main path of residual blocks).
    """
    if drop_prob == 0. or not training:
        return x
    keep_prob = 1 - drop_prob
    shape = (x.shape[0],) + (1,) * (x.ndim - 1)  # work with diff dim tensors, not just 2D ConvNets
    random_tensor = keep_prob + torch.rand(shape, dtype=x.dtype, device=x.device)
    random_tensor.floor_()  # binarize
    output = x.div(keep_prob) * random_tensor
    return output

In [6]:
def to_2tuple(x):
    if isinstance(x, tuple):
        return x
    return (x, x)

In [7]:
def trunc_normal_(tensor, mean=0., std=1., a=-2., b=2.):
    # Cut & paste from PyTorch official master until it's in a few official releases - RW
    # Method based on https://people.sc.fsu.edu/~jburkardt/presentations/truncated_normal.pdf
    def norm_cdf(x):
        # Computes standard normal cumulative distribution function
        return (1. + math.erf(x / math.sqrt(2.))) / 2.

    if (mean < a - 2 * std) or (mean > b + 2 * std):
        warnings.warn("mean is more than 2 std from [a, b] in nn.init.trunc_normal_. "
                      "The distribution of values may be incorrect.",
                      stacklevel=2)

    with torch.no_grad():
        # Values are generated within the interval [a, b].
        # (1) Generate values from the uniform distribution U(low, high).
        low = norm_cdf((a - mean) / std)
        high = norm_cdf((b - mean) / std)
        tensor.uniform_(low, high)
        # (2) Use the inverse CDF transform for the normal distribution.
        tensor.erfinv_()
        # (3) Transform to the desired mean and std
        tensor.mul_(std * math.sqrt(2.))
        tensor.add_(mean)
        # (4) Clamp to ensure values are still in the interval [a, b]
        tensor.clamp_(min=a, max=b)
        return tensor

In [8]:
class DropPath(nn.Module):
    """Drop paths (Stochastic Depth) per sample  (when applied in main path of residual blocks).
    """
    def __init__(self, drop_prob=None):
        super(DropPath, self).__init__()
        self.drop_prob = drop_prob

    def forward(self, x):
        return drop_path(x, self.drop_prob, self.training)
    
    def extra_repr(self) -> str:
        return 'p={}'.format(self.drop_prob)

class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

In [9]:
class Attention(nn.Module):
    def __init__(
            self, dim, num_heads=8, qkv_bias=False, qk_scale=None, attn_drop=0.,
            proj_drop=0., window_size=None, attn_head_dim=None):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads
        if attn_head_dim is not None:
            head_dim = attn_head_dim
        all_head_dim = head_dim * self.num_heads
        self.scale = qk_scale or head_dim ** -0.5

        self.qkv = nn.Linear(dim, all_head_dim * 3, bias=False)
        if qkv_bias:
            self.q_bias = nn.Parameter(torch.zeros(all_head_dim))
            self.v_bias = nn.Parameter(torch.zeros(all_head_dim))
        else:
            self.q_bias = None
            self.v_bias = None

        if window_size:
            self.window_size = window_size
            self.num_relative_distance = (2 * window_size[0] - 1) * (2 * window_size[1] - 1) + 3
            self.relative_position_bias_table = nn.Parameter(
                torch.zeros(self.num_relative_distance, num_heads))  # 2*Wh-1 * 2*Ww-1, nH

            # get pair-wise relative position index for each token inside the window
            coords_h = torch.arange(window_size[0])
            coords_w = torch.arange(window_size[1])
            coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
            coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
            relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
            relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
            relative_coords[:, :, 0] += window_size[0] - 1  # shift to start from 0
            relative_coords[:, :, 1] += window_size[1] - 1
            relative_coords[:, :, 0] *= 2 * window_size[1] - 1
            relative_position_index = \
                torch.zeros(size=(window_size[0] * window_size[1] + 1, ) * 2, dtype=relative_coords.dtype)
            relative_position_index[1:, 1:] = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
            relative_position_index[0, 0:] = self.num_relative_distance - 3
            relative_position_index[0:, 0] = self.num_relative_distance - 2
            relative_position_index[0, 0] = self.num_relative_distance - 1

            self.register_buffer("relative_position_index", relative_position_index)
        else:
            self.window_size = None
            self.relative_position_bias_table = None
            self.relative_position_index = None

        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(all_head_dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x, rel_pos_bias=None):
        B, N, C = x.shape
        qkv_bias = None
        if self.q_bias is not None:
            qkv_bias = torch.cat((self.q_bias, torch.zeros_like(self.v_bias, requires_grad=False), self.v_bias))
        qkv = F.linear(input=x, weight=self.qkv.weight, bias=qkv_bias)
        qkv = qkv.reshape(B, N, 3, self.num_heads, -1).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]   # make torchscript happy (cannot use tensor as tuple)

        q = q * self.scale
        attn = (q @ k.transpose(-2, -1))

        if self.relative_position_bias_table is not None:
            relative_position_bias = \
                self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
                    self.window_size[0] * self.window_size[1] + 1,
                    self.window_size[0] * self.window_size[1] + 1, -1)  # Wh*Ww,Wh*Ww,nH
            relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
            attn = attn + relative_position_bias.unsqueeze(0)

        if rel_pos_bias is not None:
            attn = attn + rel_pos_bias
        
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, -1)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [10]:
class Block(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop=0., attn_drop=0.,
                 drop_path=0., init_values=None, act_layer=nn.GELU, norm_layer=nn.LayerNorm,
                 window_size=None, attn_head_dim=None):
        super().__init__()
        self.norm1 = norm_layer(dim)
        self.attn = Attention(
            dim, num_heads=num_heads, qkv_bias=qkv_bias, qk_scale=qk_scale,
            attn_drop=attn_drop, proj_drop=drop, window_size=window_size, attn_head_dim=attn_head_dim)
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()
        self.norm2 = norm_layer(dim)
        mlp_hidden_dim = int(dim * mlp_ratio)
        self.mlp = Mlp(in_features=dim, hidden_features=mlp_hidden_dim, act_layer=act_layer, drop=drop)

        if init_values is not None:
            self.gamma_1 = nn.Parameter(init_values * torch.ones((dim)),requires_grad=True)
            self.gamma_2 = nn.Parameter(init_values * torch.ones((dim)),requires_grad=True)
        else:
            self.gamma_1, self.gamma_2 = None, None

    def forward(self, x, rel_pos_bias=None):
        if self.gamma_1 is None:
            x = x + self.drop_path(self.attn(self.norm1(x), rel_pos_bias=rel_pos_bias))
            x = x + self.drop_path(self.mlp(self.norm2(x)))
        else:
            x = x + self.drop_path(self.gamma_1 * self.attn(self.norm1(x), rel_pos_bias=rel_pos_bias))
            x = x + self.drop_path(self.gamma_2 * self.mlp(self.norm2(x)))
        return x

In [11]:
class PatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.patch_shape = (img_size[0] // patch_size[0], img_size[1] // patch_size[1])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x, **kwargs):
        B, C, H, W = x.shape
        x = self.proj(x)
        Hp, Wp = x.shape[2], x.shape[3]

        x = x.flatten(2).transpose(1, 2)
        return x, (Hp, Wp)

In [12]:
class RelativePositionBias(nn.Module):
    def __init__(self, window_size, num_heads):
        super().__init__()
        self.window_size = window_size
        self.num_relative_distance = (2 * window_size[0] - 1) * (2 * window_size[1] - 1) + 3
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros(self.num_relative_distance, num_heads))  # 2*Wh-1 * 2*Ww-1, nH

        # get pair-wise relative position index for each token inside the window
        coords_h = torch.arange(window_size[0])
        coords_w = torch.arange(window_size[1])
        coords = torch.stack(torch.meshgrid([coords_h, coords_w]))  # 2, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
        relative_coords[:, :, 0] += window_size[0] - 1  # shift to start from 0
        relative_coords[:, :, 1] += window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * window_size[1] - 1
        relative_position_index = \
            torch.zeros(size=(window_size[0] * window_size[1] + 1,) * 2, dtype=relative_coords.dtype)
        relative_position_index[1:, 1:] = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        relative_position_index[0, 0:] = self.num_relative_distance - 3
        relative_position_index[0:, 0] = self.num_relative_distance - 2
        relative_position_index[0, 0] = self.num_relative_distance - 1

        self.register_buffer("relative_position_index", relative_position_index)

    def forward(self):
        relative_position_bias = \
            self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
                self.window_size[0] * self.window_size[1] + 1,
                self.window_size[0] * self.window_size[1] + 1, -1)  # Wh*Ww,Wh*Ww,nH
        return relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww


In [13]:
class BEiT(nn.Module):
    """ Vision Transformer with support for patch or hybrid CNN input stage
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, num_classes=80, embed_dim=768, depth=12,
                 num_heads=12, mlp_ratio=4., qkv_bias=False, qk_scale=None, drop_rate=0., attn_drop_rate=0.,
                 drop_path_rate=0., norm_layer=None, init_values=None, use_checkpoint=False, 
                 use_abs_pos_emb=True, use_rel_pos_bias=False, use_shared_rel_pos_bias=False,
                 out_indices=[3, 5, 7, 11]):
        super().__init__()
        norm_layer = norm_layer or partial(nn.LayerNorm, eps=1e-6)
        self.num_classes = num_classes
        self.num_features = self.embed_dim = embed_dim  # num_features for consistency with other models

        self.patch_embed = PatchEmbed(
            img_size=img_size, patch_size=patch_size, in_chans=in_chans, embed_dim=embed_dim)
        num_patches = self.patch_embed.num_patches
        self.out_indices = out_indices

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        if use_abs_pos_emb:
            self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        else:
            self.pos_embed = None
        self.pos_drop = nn.Dropout(p=drop_rate)

        if use_shared_rel_pos_bias:
            self.rel_pos_bias = RelativePositionBias(window_size=self.patch_embed.patch_shape, num_heads=num_heads)
        else:
            self.rel_pos_bias = None

        dpr = [x.item() for x in torch.linspace(0, drop_path_rate, depth)]  # stochastic depth decay rule
        self.use_rel_pos_bias = use_rel_pos_bias
        self.use_checkpoint = use_checkpoint
        self.blocks = nn.ModuleList([
            Block(
                dim=embed_dim, num_heads=num_heads, mlp_ratio=mlp_ratio, qkv_bias=qkv_bias, qk_scale=qk_scale,
                drop=drop_rate, attn_drop=attn_drop_rate, drop_path=dpr[i], norm_layer=norm_layer,
                init_values=init_values, window_size=self.patch_embed.patch_shape if use_rel_pos_bias else None)
            for i in range(depth)])

        if self.pos_embed is not None:
            trunc_normal_(self.pos_embed, std=.02)
        trunc_normal_(self.cls_token, std=.02)
        self.out_indices = out_indices

        if patch_size == 16:
            self.fpn1 = nn.Sequential(
                nn.ConvTranspose2d(embed_dim, embed_dim, kernel_size=2, stride=2),
                nn.BatchNorm2d(embed_dim),
                nn.GELU(),
                nn.ConvTranspose2d(embed_dim, embed_dim, kernel_size=2, stride=2),
            )

            self.fpn2 = nn.Sequential(
                nn.ConvTranspose2d(embed_dim, embed_dim, kernel_size=2, stride=2),
            )

            self.fpn3 = nn.Identity()

            self.fpn4 = nn.MaxPool2d(kernel_size=2, stride=2)
        elif patch_size == 8:
            self.fpn1 = nn.Sequential(
                nn.ConvTranspose2d(embed_dim, embed_dim, kernel_size=2, stride=2),
            )

            self.fpn2 = nn.Identity()

            self.fpn3 = nn.Sequential(
                nn.MaxPool2d(kernel_size=2, stride=2),
            )

            self.fpn4 = nn.Sequential(
                nn.MaxPool2d(kernel_size=4, stride=4),
            )
        self.apply(self._init_weights)
        self.fix_init_weight()

    def fix_init_weight(self):
        def rescale(param, layer_id):
            param.div_(math.sqrt(2.0 * layer_id))

        for layer_id, layer in enumerate(self.blocks):
            rescale(layer.attn.proj.weight.data, layer_id + 1)
            rescale(layer.mlp.fc2.weight.data, layer_id + 1)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            trunc_normal_(m.weight, std=.02)
            if isinstance(m, nn.Linear) and m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def get_num_layers(self):
        return len(self.blocks)

    def forward_features(self, x):
        B, C, H, W = x.shape
        x, (Hp, Wp) = self.patch_embed(x)
        batch_size, seq_len, _ = x.size()

        cls_tokens = self.cls_token.expand(batch_size, -1, -1)  # stole cls_tokens impl from Phil Wang, thanks
        x = torch.cat((cls_tokens, x), dim=1)
        if self.pos_embed is not None:
            x = x + self.pos_embed
        x = self.pos_drop(x)

        rel_pos_bias = self.rel_pos_bias() if self.rel_pos_bias is not None else None
        features = []
        for i, blk in enumerate(self.blocks):
            if self.use_checkpoint:
                x = checkpoint.checkpoint(blk, x, rel_pos_bias)
            else:
                x = blk(x, rel_pos_bias)
            if i in self.out_indices:
                xp = x[:, 1:, :].permute(0, 2, 1).reshape(B, -1, Hp, Wp)
                features.append(xp.contiguous())

        ops = [self.fpn1, self.fpn2, self.fpn3, self.fpn4]
        for i in range(len(features)):
            features[i] = ops[i](features[i])

        return tuple(features)

    def forward(self, x):
        x = self.forward_features(x)
        return x


# **Segmentation Model** 

In [14]:
# Define SegmentationHead
class SegmentationHead(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, num_classes, kernel_size=1)
        
    def forward(self, x):
        return self.conv(x)

In [15]:
# Define the BEiT-based Semantic Segmentation model
class BEiTSegmentation(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # Calculate patch dimensions for 256x256 input
        img_size = 256  # Match transform size
        patch_size = 16
        num_patches = (img_size // patch_size) ** 2  # 256/16 = 16 patches per side
        
        self.backbone = BEiT(
            img_size=img_size,       # Changed to match transform size
            patch_size=patch_size,
            in_chans=3,
            embed_dim=512,
            depth=6,
            num_heads=8,
            mlp_ratio=4.,
            qkv_bias=True,
            use_abs_pos_emb=True,
            use_rel_pos_bias=False,
            init_values=0.1,
            drop_path_rate=0.1,
            out_indices=[2, 3, 4, 5],
            use_checkpoint=True
        )
        
        # Adjust decoder for 256x256 input
        self.decoder = nn.Sequential(
            nn.Conv2d(512, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(256, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        )
        
        self.segmentation_head = SegmentationHead(64, num_classes)
        
    def forward(self, x):
        features = self.backbone(x)
        x = features[-1]
        x = self.decoder(x)
        x = self.segmentation_head(x)
        # Interpolate to input size
        x = nn.functional.interpolate(x, size=(256, 256), mode='bilinear', align_corners=True)
        return x

# **Dataset and Training Functions**

In [16]:
#---------------  ---------------#

# Load dataset configuration
def load_data_config(yaml_path):
    """Load dataset configuration from YAML file"""
    with open(yaml_path, 'r') as f:
        config = yaml.safe_load(f)
    return config

In [17]:
# Custom Dataset for semantic segmentation
class TumorSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, num_classes=3):  # Add num_classes parameter
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.num_classes = num_classes  # Store num_classes
        
        # Support both extensions
        self.image_files = [f for f in os.listdir(image_dir) 
                          if f.endswith(('.jpg', '.png', '.jpeg'))]
        
        # For each image, find the corresponding mask
        self.valid_pairs = []
        for img_file in self.image_files:
            img_name = os.path.splitext(img_file)[0]
            mask_file = None
            
            # Look for masks with different extensions
            for ext in ['.txt', '.png', '.jpg']:
                candidate = os.path.join(mask_dir, img_name + ext)
                if os.path.exists(candidate):
                    mask_file = img_name + ext
                    break
            
            if mask_file:
                self.valid_pairs.append((img_file, mask_file))
        
        print(f"Found {len(self.valid_pairs)} valid image-mask pairs out of {len(self.image_files)} images")
        
        if not self.valid_pairs:
            print("WARNING: No valid image-mask pairs found!")
            print("This will cause an error with DataLoader.")

    def __len__(self):
        return len(self.valid_pairs)
    
    def __getitem__(self, idx):
        img_file, mask_file = self.valid_pairs[idx]
        
        # Load image
        img_path = os.path.join(self.image_dir, img_file)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load mask
        mask_path = os.path.join(self.mask_dir, mask_file)
        
        # Handle different mask formats
        if mask_file.endswith('.txt'):  # YOLO format
            h, w = image.shape[:2]
            mask = np.zeros((h, w), dtype=np.int64)
            
            try:
                with open(mask_path, 'r') as f:
                    lines = f.readlines()
                
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:
                        class_id = int(parts[0])
                        # Ensure class_id is valid
                        if class_id >= self.num_classes:
                            print(f"Warning: Invalid class ID {class_id} in {mask_path}")
                            continue
                        
                        x_center, y_center, bbox_width, bbox_height = map(float, parts[1:5])
                        
                        x1 = max(0, int((x_center - bbox_width/2) * w))
                        y1 = max(0, int((y_center - bbox_height/2) * h))
                        x2 = min(w-1, int((x_center + bbox_width/2) * w))
                        y2 = min(h-1, int((y_center + bbox_height/2) * h))
                        
                        mask[y1:y2+1, x1:x2+1] = class_id  # Remove +1 offset
            except Exception as e:
                print(f"Error processing mask {mask_path}: {e}")
                mask = np.zeros((h, w), dtype=np.int64)
        else:  # Image format
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = mask.astype(np.int64)
            # Ensure mask values are valid class indices
            mask[mask >= self.num_classes] = 0
        
        # Apply transformations
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask'].long()
        
        return image, mask

In [18]:
# Training function
def train_one_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    
    with tqdm(dataloader, desc="Training") as pbar:
        for images, masks in pbar:
            images = images.to(device, non_blocking=True)
            masks = masks.to(device, non_blocking=True)
            
            # Clear gradients
            optimizer.zero_grad(set_to_none=True)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            # Backward pass
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            
            total_loss += loss.item()
            pbar.set_postfix(loss=loss.item())
            
            # Clear cache every few iterations
            if pbar.n % 10 == 0:
                torch.cuda.empty_cache()
    
    return total_loss / len(dataloader)

In [19]:
# Validation function
def validate(model, dataloader, criterion, device, num_classes):
    model.eval()
    total_loss = 0
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        with tqdm(dataloader, desc="Validating") as pbar:
            for images, masks in pbar:
                images = images.to(device)
                masks = masks.to(device)
                
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, masks)
                
                total_loss += loss.item()
                
                # Get predictions
                preds = torch.argmax(outputs, dim=1).cpu().numpy()
                targets = masks.cpu().numpy()
                
                all_preds.extend(preds.flatten())
                all_targets.extend(targets.flatten())
                
                pbar.set_postfix(loss=loss.item())
    
    # Calculate IoU for each class
    iou_per_class = []
    for cls in range(num_classes):
        pred_cls = np.array(all_preds) == cls
        target_cls = np.array(all_targets) == cls
        intersection = np.logical_and(pred_cls, target_cls).sum()
        union = np.logical_or(pred_cls, target_cls).sum()
        if union == 0:
            iou = float('nan')
        else:
            iou = intersection / union
        iou_per_class.append(iou)
    mean_iou = np.nanmean(iou_per_class)
    return total_loss / len(dataloader), mean_iou, iou_per_class

In [20]:
# Visualization function
def visualize_prediction(model, test_img_path, device, class_names):
    """Visualize prediction on a single test image with improved overlay"""
    # Prepare image
    image = cv2.imread(test_img_path)
    if image is None:
        print(f"Error loading image: {test_img_path}")
        return
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Store original dimensions
    orig_h, orig_w = image.shape[:2]
    
    # Resize image to model input size
    target_size = (256, 256)
    resized_image = cv2.resize(image, target_size)
    
    # Apply transforms
    transform = A.Compose([
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])
    
    # Transform and add batch dimension
    transformed = transform(image=resized_image)
    input_tensor = transformed['image'].unsqueeze(0).to(device)
    
    # Get prediction
    model.eval()
    with torch.no_grad():
        output = model(input_tensor)
        pred_mask = torch.argmax(output, dim=1)[0].cpu().numpy()
    
    # Resize prediction mask back to original image size
    pred_mask = cv2.resize(pred_mask.astype(float), (orig_w, orig_h), 
                          interpolation=cv2.INTER_NEAREST).astype(int)
    
    # Create figure
    plt.figure(figsize=(15, 5))
    
    # 1. Original image
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title('Original Image')
    plt.axis('off')
    
    # 2. Prediction mask
    plt.subplot(1, 3, 2)
    # Create a colored mask
    mask_display = np.zeros((*pred_mask.shape, 3), dtype=np.uint8)
    tumor_regions = (pred_mask == 1)
    mask_display[tumor_regions] = [255, 0, 0]  # Red for tumor
    plt.imshow(mask_display)
    plt.title('Predicted Segmentation')
    plt.axis('off')
    
    # 3. Overlay
    plt.subplot(1, 3, 3)
    overlay = image.copy()
    
    # Only create overlay if tumor regions are detected
    if np.any(tumor_regions):
        # Create tumor overlay
        mask_overlay = np.zeros_like(image)
        mask_overlay[tumor_regions] = [255, 0, 0]
        
        # Blend images
        alpha = 0.5
        overlay = cv2.addWeighted(image, 1, mask_overlay, alpha, 0)
        
        # Add contours
        tumor_mask_uint8 = tumor_regions.astype(np.uint8)
        contours, _ = cv2.findContours(tumor_mask_uint8, 
                                     cv2.RETR_EXTERNAL, 
                                     cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(overlay, contours, -1, (255, 255, 255), 2)
    
    plt.imshow(overlay)
    plt.title('Overlay with Tumor Regions' if np.any(tumor_regions) else 'No Tumor Detected')
    plt.axis('off')
    
    # Save with high quality
    plt.tight_layout()
    plt.savefig(f'prediction_epoch_{epoch+1}.png', 
                dpi=300, 
                bbox_inches='tight',
                pad_inches=0.1)
    plt.close()

    # Print detection status
    if not np.any(tumor_regions):
        print(f"No tumor regions detected in {os.path.basename(test_img_path)}")

# **Loading Dataset**

In [21]:
# Load config
config = load_data_config("/kaggle/input/brain-tumor-dataset/brain tumor.v2-release.yolov7pytorch/data.yaml")
num_classes = config['nc']
class_names = config['names']
print(f"Number of classes: {num_classes}")
print(f"Class names: {class_names}")

Number of classes: 3
Class names: ['label0', 'label1', 'label2']


In [22]:
# Setup paths relative to yaml location
base_dir = os.path.dirname("/kaggle/input/brain-tumor-dataset/brain tumor.v2-release.yolov7pytorch/data.yaml")
train_img_dir = os.path.join(base_dir, "train/images")
train_mask_dir = os.path.join(base_dir, "train/labels")
val_img_dir = os.path.join(base_dir, "valid/images")
val_mask_dir = os.path.join(base_dir, "valid/labels")

print(f"Using device: {device}")
check_directory_structure("/kaggle/input/brain-tumor-dataset/brain tumor.v2-release.yolov7pytorch")

Using device: cuda

--- Directory Structure Check for /kaggle/input/brain-tumor-dataset/brain tumor.v2-release.yolov7pytorch ---
Directory /kaggle/input/brain-tumor-dataset/brain tumor.v2-release.yolov7pytorch/train/images: Exists, Contains 6930 files
  Example files: ['volume_66_slice_107_jpg.rf.3323c23ffb4eccdd843337b27556bd61.jpg', 'volume_248_slice_71_jpg.rf.24606413467e216b0ef9c1bae9e52280.jpg', 'volume_342_slice_69_jpg.rf.fd98a1f55f0e5a10ccf73e39c48be04b.jpg']
Directory /kaggle/input/brain-tumor-dataset/brain tumor.v2-release.yolov7pytorch/train/labels: Exists, Contains 6930 files
  Example files: ['volume_355_slice_90_jpg.rf.3329d69004d7cfcde34e54498d57c092.txt', 'volume_27_slice_32_jpg.rf.f8a5cbf01d0b04abae74dfa30fb00f98.txt', 'volume_172_slice_102_jpg.rf.1e354849bb2dac58094a1d147adf833a.txt']
Directory /kaggle/input/brain-tumor-dataset/brain tumor.v2-release.yolov7pytorch/valid/images: Exists, Contains 1980 files
  Example files: ['volume_362_slice_52_jpg.rf.ae6c7adf7d668c1777

In [23]:
# Transforms with smaller image size
train_transform = A.Compose([
    A.Resize(256, 256),  # Reduced from 384
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(),
    ToTensorV2()
])

In [24]:
val_transform = A.Compose([
    A.Resize(256, 256),  # Reduced from 384
    A.Normalize(),
    ToTensorV2()
])

In [25]:
# Datasets and loaders
train_dataset = TumorSegmentationDataset(
    train_img_dir, 
    train_mask_dir, 
    transform=train_transform,
    num_classes=num_classes  # Now using 3 classes from yaml
)

Found 6930 valid image-mask pairs out of 6930 images


In [26]:
val_dataset = TumorSegmentationDataset(
    val_img_dir, 
    val_mask_dir, 
    transform=val_transform,
    num_classes=num_classes
)

Found 1980 valid image-mask pairs out of 1980 images


In [27]:
# Add validation before training
print("\nValidating class indices...")
max_train = max(mask.max() for _, mask in train_dataset)
max_val = max(mask.max() for _, mask in val_dataset)
print(f"Max class index in train: {max_train}")
print(f"Max class index in val: {max_val}")
print(f"Number of classes: {num_classes}")


Validating class indices...
Max class index in train: 2
Max class index in val: 2
Number of classes: 3


In [28]:
assert max_train < num_classes, f"Invalid class index in train set: {max_train} >= {num_classes}"
assert max_val < num_classes, f"Invalid class index in val set: {max_val} >= {num_classes}"

In [29]:
# Validate class indices
print("\nValidating mask values...")
for dataset, name in [(train_dataset, 'train'), (val_dataset, 'val')]:
    unique_classes = set()
    for _, mask in dataset:
        unique_classes.update(mask.unique().tolist())
    print(f"{name} dataset unique classes: {sorted(unique_classes)}")
    assert max(unique_classes) < num_classes, \
        f"Invalid class index in {name} set: {max(unique_classes)} >= {num_classes}"


Validating mask values...
train dataset unique classes: [0, 1, 2]
val dataset unique classes: [0, 1, 2]


In [30]:
print("\nDataset Statistics:")
print("-" * 50)
print(f"Images in train directory: {len(os.listdir(train_img_dir))}")
print(f"Masks in train directory: {len(os.listdir(train_mask_dir))}")
print("\nSample image-mask pairs:")
print("-" * 50)
print("First 5 train pairs:", train_dataset.valid_pairs[:5])
print(f"\nTotal valid pairs:")
print("-" * 50)
print(f"Train dataset: {len(train_dataset)} pairs")
print(f"Val dataset: {len(val_dataset)} pairs")


Dataset Statistics:
--------------------------------------------------
Images in train directory: 6930
Masks in train directory: 6930

Sample image-mask pairs:
--------------------------------------------------
First 5 train pairs: [('volume_66_slice_107_jpg.rf.3323c23ffb4eccdd843337b27556bd61.jpg', 'volume_66_slice_107_jpg.rf.3323c23ffb4eccdd843337b27556bd61.txt'), ('volume_248_slice_71_jpg.rf.24606413467e216b0ef9c1bae9e52280.jpg', 'volume_248_slice_71_jpg.rf.24606413467e216b0ef9c1bae9e52280.txt'), ('volume_342_slice_69_jpg.rf.fd98a1f55f0e5a10ccf73e39c48be04b.jpg', 'volume_342_slice_69_jpg.rf.fd98a1f55f0e5a10ccf73e39c48be04b.txt'), ('volume_175_slice_75_jpg.rf.e8283a8e19d28d1a3bc5f8632a829255.jpg', 'volume_175_slice_75_jpg.rf.e8283a8e19d28d1a3bc5f8632a829255.txt'), ('volume_85_slice_107_jpg.rf.c6cb235444db622026deff749844f043.jpg', 'volume_85_slice_107_jpg.rf.c6cb235444db622026deff749844f043.txt')]

Total valid pairs:
--------------------------------------------------
Train dataset: 

In [31]:
# Model, loss, optimizer
model = BEiTSegmentation(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.01)

In [32]:
# Create data loaders with smaller batch size
train_loader = DataLoader(
    train_dataset, 
    batch_size=1,       # Reduced from 2
    shuffle=True, 
    num_workers=0,      # Reduced from 2
    pin_memory=True     # Add pin_memory
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=1,       # Reduced from 2
    shuffle=True, 
    num_workers=0,      # Reduced from 2
    pin_memory=True     # Add pin_memory
)

In [33]:
# Get list of test images
test_img_dir = os.path.join(base_dir, "test/images")
test_images = [os.path.join(test_img_dir, f) for f in os.listdir(test_img_dir) 
              if f.endswith(('.jpg', '.png', '.jpeg'))]
if not test_images:
    print("Warning: No test images found!")

# **Training**

In [34]:
# Training loop
best_miou = 0
num_epochs = 60
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    
    # Clear memory
    torch.cuda.empty_cache()
    
    val_loss, val_miou, val_ious = validate(model, val_loader, criterion, device, num_classes)
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | mIoU: {val_miou:.4f}")
    print(f"IoU per class: {val_ious}")

    # Visualize prediction on random test image
    if test_images:
        random_test_img = random.choice(test_images)
        print(f"\nGenerating prediction visualization for: {os.path.basename(random_test_img)}")
        visualize_prediction(model, random_test_img, device, class_names)

    # Save best model
    if val_miou > best_miou:
        best_miou = val_miou
        torch.save(model.state_dict(), "best_beit_segmentation.pth")
        print("Best model saved.")
    
    # Clear memory again
    torch.cuda.empty_cache()

print("Training complete.")


Epoch 1/60


Validating: 100%|██████████| 1980/1980 [00:59<00:00, 33.10it/s, loss=0.596]


Train Loss: 0.1452 | Val Loss: 0.1959 | mIoU: 0.4267
IoU per class: [0.9399134095177615, 0.18364479923855997, 0.1566250466033696]

Generating prediction visualization for: volume_10_slice_91_jpg.rf.b4a84f6c1b5ffcd12fc80f3b99c496f6.jpg
No tumor regions detected in volume_10_slice_91_jpg.rf.b4a84f6c1b5ffcd12fc80f3b99c496f6.jpg
Best model saved.

Epoch 2/60


Validating: 100%|██████████| 1980/1980 [00:58<00:00, 33.66it/s, loss=0.0621]


Train Loss: 0.0957 | Val Loss: 0.1688 | mIoU: 0.4474
IoU per class: [0.9587347179656114, 0.16876809755029581, 0.21461161820073113]

Generating prediction visualization for: volume_344_slice_77_jpg.rf.d29c3f0fd784a1365c83e3964539a1aa.jpg
Best model saved.

Epoch 3/60


...

Train Loss: 0.0174 | Val Loss: 0.1358 | mIoU: 0.6349
IoU per class: [0.97317472001813, 0.43329741026875923, 0.49818921564096735]

Generating prediction visualization for: volume_305_slice_134_jpg.rf.8a9645e5ccfb9a7e8945c1efe8284905.jpg

Epoch 59/60


Validating: 100%|██████████| 1980/1980 [00:57<00:00, 34.53it/s, loss=0.336]


Train Loss: 0.0173 | Val Loss: 0.1518 | mIoU: 0.6227
IoU per class: [0.9708860793776645, 0.41928676977253077, 0.47797342372928814]

Generating prediction visualization for: volume_203_slice_36_jpg.rf.9d1e4c29df3087c3c5802a7ef67310a5.jpg

Epoch 60/60


Validating: 100%|██████████| 1980/1980 [00:59<00:00, 33.42it/s, loss=0.125]


Train Loss: 0.0170 | Val Loss: 0.1346 | mIoU: 0.6329
IoU per class: [0.9737531107929562, 0.45488732897085066, 0.4700601196464046]

Generating prediction visualization for: volume_221_slice_40_jpg.rf.96f99038840825da52449f8039f4a006.jpg
Training complete.
